in this note book we implement excercise 2.


we will be working interactivley:

load variables into global and interact in jupyter. only things work and are understood, do we move to classes and fucntions

# imports and setup

In [ ]:
import sys, os
sys.path.append(os.getcwd())

In [ ]:
import numpy as np
import faiss
from helpers import *

## brief summary of methods in helpers

**Helper functions:**
- `decision_tree_demo()`: Demo of training/predicting with DecisionTreeClassifier
- `loading_random_forest()`: Shows how to initialize RandomForestClassifier with 300 trees, max_depth=6
- `loading_xgboost()`: Shows how to initialize XGBClassifier with 300 trees, max_depth=6, lr=0.1
- `plot_decision_boundaries(model, X, y, title)`: Visualizes classifier decision boundaries on 2D space
- `knn_examples(X_train, Y_train, X_test, Y_test)`: Demo of training/predicting with KNNClassifier
- `read_data_demo(filename)`: Reads CSV and returns numpy array + column names

## data overview

**Task:** Create paths to all CSV files (train.csv, validation.csv, test.csv, AD_test.csv). Print first few rows of train and AD_test. Print shapes of all datasets.

help me create heper vars holding paths to data files and to visualise fist few enteries of data and their shape

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

```python
from pathlib import Path

data_dir = Path('data')
train_path = data_dir/'train.csv'
val_path = data_dir/'validation.csv'
test_path = data_dir/'test.csv'
ad_test_path = data_dir/'AD_test.csv'

train_path, val_path, test_path, ad_test_path
```

Once you run that, we can load and visualize the data:

```python
train_data, train_cols = read_data_demo(train_path)
ad_test_data, ad_test_cols = read_data_demo(ad_test_path)

print(f"Train data first 5 rows:\n{train_data[:5]}\n")
print(f"Train columns: {train_cols}\n")
print(f"AD_test data first 5 rows:\n{ad_test_data[:5]}\n")
print(f"AD_test columns: {ad_test_cols}")
```

Then check all shapes:

```python
val_data, _ = read_data_demo(val_path)
test_data, _ = read_data_demo(test_path)

print(f"Train shape: {train_data.shape}")
print(f"Validation shape: {val_data.shape}")
print(f"Test shape: {test_data.shape}")
print(f"AD_test shape: {ad_test_data.shape}")
```

In [ ]:
from pathlib import Path

data_dir = Path('data')
train_path = data_dir/'train.csv'
val_path = data_dir/'validation.csv'
test_path = data_dir/'test.csv'
ad_test_path = data_dir/'AD_test.csv'

train_path, val_path, test_path, ad_test_path

(Path('data/train.csv'),
 Path('data/validation.csv'),
 Path('data/test.csv'),
 Path('data/AD_test.csv'))

In [ ]:
train_data, train_cols = read_data_demo(train_path)
ad_test_data, ad_test_cols = read_data_demo(ad_test_path)

print(f"Train data first 5 rows:\n{train_data[:5]}\n")
print(f"Train columns: {train_cols}\n")
print(f"AD_test data first 5 rows:\n{ad_test_data[:5]}\n")
print(f"AD_test columns: {ad_test_cols}")

FileNotFoundError: [Errno 2] No such file or directory: 'data/train.csv'


check current dir and its contents

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

```python
Path.cwd()
```

```python
list(Path.cwd().iterdir())
```

In [ ]:
val_data, _ = read_data_demo(val_path)
test_data, _ = read_data_demo(test_path)

print(f"Train shape: {train_data.shape}")
print(f"Validation shape: {val_data.shape}")
print(f"Test shape: {test_data.shape}")
print(f"AD_test shape: {ad_test_data.shape}")

# part 1: KNN iplementation

specification notes

Algorithm 1: kNN Classification

Data: Training dataset Xtrain = {(xi, yi)}N

i=1, where xi ∈ R2 is the

feature vector and yi corresponds to its class label; Test instance
ˆxtest; Number of neighbors k.

Result: Predicted class label ˆytest for the test instance.
for each training instance (xi, yi) do

Calculate the distance di between xi and xtest.

Sort the distances di in ascending order and select the first k instances.
for each class c do

Count the occurrences of c in the selected k instances.
Assign the class label ˆytest to the one with the highest count.
return ˆytest




class KNNClassifier:
    def __init__(self, k, distance_metric='l2'):
        self.k = k
        self.distance_metric = distance_metric
        self.X_train = None
        self.Y_train = None

    def fit(self, X_train, Y_train):
        """
        Update the kNN classifier with the provided training data.

        Parameters:
        - X_train (numpy array) of size (N, d): Training feature vectors.
        - Y_train (numpy array) of size (N,): Corresponding class labels.
        """
        self.X_train = X_train.astype(np.float32)
        self.Y_train = Y_train
        d = self.X_train.shape[1]
        if self.distance_metric == 'l2':
            self.index = faiss.index_factory(d, "Flat", faiss.METRIC_L2)
        elif self.distance_metric == 'l1':
            self.index = faiss.index_factory(d, "Flat", faiss.METRIC_L1)
        else:
            raise NotImplementedError
        pass
        self.index.add(self.X_train)

    def predict(self, X):
        """
        Predict the class labels for the given data.

        Parameters:
        - X (numpy array) of size (M, d): Feature vectors.

        Returns:
        - (numpy array) of size (M,): Predicted class labels.
        """
        #### YOUR CODE GOES HERE ####

    def knn_distance(self, X):
        """
        Calculate kNN distances for the given data. You must use the faiss library to compute the distances.
        See lecture slides and https://github.com/facebookresearch/faiss/wiki/Getting-started#in-python-2 for more information.

        Parameters:
        - X (numpy array) of size (M, d): Feature vectors.

        Returns:
        - (numpy array) of size (M, k): kNN distances.
        - (numpy array) of size (M, k): Indices of kNNs.
        """
        X = X.astype(np.float32)
	#### YOUR CODE GOES HERE ####


things i need to understand:

how does faiss knn work? what happens in fit?
we need to be able to play with the internals


notes: since our data lives in 2 d, can we use heuristics to pull top k closest neightbours without going over all data?

create some kind of lookup bin

## KNN grid search

**Grid search:** Train kNN for all combinations of k ∈ {1, 10, 100, 1000, 3000} × distance ∈ {L1, L2}. Create 5×2 table of test accuracies. Save all 10 models with their hyperparameters and test accuracies.

**Questions:**
1. What's the trend as k increases? Does it differ by distance metric?
2. Visualize decision boundaries for: (i) L2 with k_max (best accuracy), (ii) L2 with k_min (worst accuracy), (iii) L1 with k_max
   - (a) Compare k_max vs k_min with L2: why does k_max perform better?
   - (b) Compare L2 vs L1 with k_max: how does distance metric affect the space?

think about the problem - what would you expect?

### KNN decision boundaries


**Task:** Use `plot_decision_boundaries()` helper to visualize 3 models on test data:
1. L2 distance, k = k_max (highest test accuracy)
2. L2 distance, k = k_min (lowest test accuracy)  
3. L1 distance, k = k_max

## Anomaly detection

**Task:** Use AD_test.csv as test set, train.csv as single class (ignore labels).
1. Find 5 nearest neighbors (L2 distance) for each AD_test sample
2. Sum the 5 distances = anomaly score per sample
3. Select top 50 samples with highest scores = anomalies, rest = normal
4. Plot: AD_test colored by prediction (blue=normal, red=anomaly) + train.csv in black with alpha=0.01

**Question:** What characterizes the anomalies? How do they differ from normal data?

# trees

## Understanding trees - interactive exploration

**Goal:** Interact with tree components to understand how predictions work.
- Create a simple tree, examine its structure (tree_.feature, tree_.threshold, tree_.children_left/right, tree_.value)
- Manually trace a prediction path for a sample point
- Understand how tree.predict() navigates the tree structure

**Visualization library:** `sklearn.tree.plot_tree()` or `graphviz` for tree visualization. For more advanced viz, use `dtreeviz` library.

In [ ]:
# Tree visualization series placeholder

## Visualizing tree complexity

**Series 1 - Fixed depth=1, varying leaves:**
Train and visualize trees with max_depth=1 and max_leaf_nodes ∈ {2, 4, 8, 24, 50, 100}

**Series 2 - Varying depth, controlled leaves:**
Train and visualize trees with max_depth ∈ {2, 4, 8, 16, 32, 50} and appropriate max_leaf_nodes to observe depth effects

**Goal:** Understand how depth vs leaf constraints affect tree structure and decision boundaries.

**Visualization:** Use `sklearn.tree.plot_tree()` for tree structure or `plot_decision_boundaries()` for geographic space.

## train 24 trees

**Task:** Train DecisionTreeClassifier for all combinations of max_depth ∈ {1, 2, 4, 6, 10, 20, 50, 100} × max_leaf_nodes ∈ {50, 100, 1000}. Save all 24 models with their hyperparameters (max_depth, max_leaf_nodes) and accuracies (train, validation, test).

## tree analysis

**Questions:**
1. Report test accuracy of tree with best validation accuracy
2. Does this tree generalize well? Is validation set sufficient for selection?
3. Are 50 leaf nodes enough for perfect accuracy with 50 states? Why/why not?
4. Visualize decision boundaries for best tree from Q1. What shapes do classes form?
5. Find best tree with exactly 50 leaf nodes. Visualize and compare to Q1 best tree.
6. Find best tree with max_depth ≤ 6. Visualize and compare space division to Q4.

think and research - how many leaves do trees? how many boundaries can a tree make? how many hyper planes?

ask chat gpt for deep think

## random forest

**Task:** Train RandomForestClassifier with n_estimators=300, max_depth=6. Visualize decision boundaries. 

**Question:** Is this model more expressive than single tree from Q1? How does visualization show this?

# bonus: boosted trees

**Bonus (5 pts):** Train XGBClassifier with n_estimators=300, max_depth=6, learning_rate=0.1. Report test accuracy and visualize predictions. Compare to random forest: how do predictions differ? Which performs better?

## interpret Ml explainable boosting classifier

instead of xgb trees, use the glassbox EBM.